In [1]:
import pandas as pd
import psycopg2

In [2]:
def create_database():
    try:
        conn=psycopg2.connect("host=localhost dbname=postgres user=postgres password=1234")
    except psycopg2.Error as e:
        print (e)

    conn.set_session(autocommit=True)
    cur=conn.cursor()
    cur.execute("DROP DATABASE accounts")
    cur.execute("CREATE DATABASE accounts")

    conn.close()
    try:
        conn=psycopg2.connect("host=localhost dbname=accounts user=postgres password=1234")
    except psycopg2.Error as e:
        print (e)
    cur=conn.cursor()

    return cur, conn

In [11]:
cur,conn=create_database()

In [25]:
def establish_conn():
    try:
            conn=psycopg2.connect("host=localhost dbname=accounts user=postgres password=1234")
    except psycopg2.Error as e:
            print (e)
    cur=conn.cursor()

    return cur, conn

In [53]:
cur,conn=establish_conn()

In [12]:
country_data_queries=("""CREATE TABLE IF NOT EXISTS accountscountry(
                      country_code VARCHAR PRIMARY KEY,
                      short_name VARCHAR,
                      table_name VARCHAR,
                      long_name VARCHAR,
                      currency_unit VARCHAR)""")

In [13]:
cur.execute(country_data_queries)
conn.commit()

In [21]:
data_queries=("""CREATE TABLE IF NOT EXISTS accountsdata(
              country_name VARCHAR,
              country_code VARCHAR,
              series_name VARCHAR,
              series_code VARCHAR,
              year_2015 numeric,
              year_2016 numeric,
              year_2017 numeric,
              year_2018 numeric)""")

In [18]:
cur.execute(data_queries)
conn.commit()

In [23]:
series_queries=("""CREATE TABLE IF NOT EXISTS accountsseries(
              country_name VARCHAR,
              code VARCHAR,
              series_name VARCHAR,
              long_name VARCHAR,
              source VARCHAR,
              topic VARCHAR)
              """)

In [27]:
cur.execute(series_queries)
conn.commit()

In [52]:
accountscountry=pd.read_csv("accountscountry.csv")
accountsdata=pd.read_csv("accountsdata.csv")
accountsseries=pd.read_csv("accountsseries.csv")

In [34]:
accountscountry.head()

,Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso
2,ARM,Armenia,Armenia,Republic of Armenia,Armenian dram
3,AUS,Australia,Australia,Commonwealth of Australia,Australian dollar
4,AUT,Austria,Austria,Republic of Austria,Euro


In [32]:
country_insert_queries=("""INSERT INTO accountscountry(
                      country_code,
                      short_name,
                      table_name,
                      long_name,
                      currency_unit)
                        VALUES (%s,%s,%s,%s,%s)""")

In [36]:
for i,row in accountscountry.iterrows():
    cur.execute(country_insert_queries,list(row))


In [38]:
conn.commit()

In [41]:
data_insert_queries=("""INSERT INTO accountsdata(
                      country_name,
              country_code,
              series_name,
              series_code,
              year_2015,
              year_2016,
              year_2017,
              year_2018)
                        VALUES (%s,%s,%s,%s,%s,%s,%s,%s)""")

In [54]:
for i,row in accountsdata.iterrows():
    cur.execute(data_insert_queries,list(row))
conn.commit()    

In [58]:
cur.execute("""ALTER TABLE accountsseries
DROP COLUMN country_name""")

In [59]:
conn.commit()

In [63]:
series_insert_queries=("""INSERT INTO accountsseries(
              code,
              series_name,
              long_name,
              source,
              topic)VALUES (%s,%s,%s,%s,%s)""")

In [64]:
for i,row in accountsseries.iterrows():
    cur.execute(series_insert_queries,list(row))
conn.commit()    